In [1]:
!pip install torch pytorch-lightning pytorch-forecasting pandas numpy requests yfinance matplotlib


[notice] A new release of pip is available: 24.0 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import warnings
import ssl
import requests
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt

# Bibliotecas de Deep Learning
import torch
import pytorch_lightning as pl
from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer, Baseline, QuantileLoss
from pytorch_forecasting.data import GroupNormalizer

# ==============================================================================
# CONFIGURAÇÃO DE AMBIENTE CORPORATIVO (BYPASS SSL)
# ==============================================================================
# Desabilita avisos de segurança para permitir download do BCB/Yahoo
warnings.filterwarnings("ignore")
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

# Configura Seeds para reprodutibilidade
pl.seed_everything(42)

c:\Users\A515110\Desktop\Repositorio Git\Projetos pessoais\venv\Lib\site-packages\pytorch_forecasting\models\base\_base_model.py:30: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm
Seed set to 42


42

# Carga e Tratamento dos Dados Granulares (Painel)

In [ ]:
# ==============================================================================
# 1. CARGA E LIMPEZA INICIAL (Parquet)
# ==============================================================================
# Lê os parquets salvos no main.ipynb e recria df_retail_transformed

import os

parquet_retail = "df_retail.parquet"
parquet_dim = "df_dim_vehicle.parquet"

if 'df_retail_transformed' in locals():
    df = df_retail_transformed.copy()
else:
    if os.path.exists(parquet_retail) and os.path.exists(parquet_dim):
        df_retail = pd.read_parquet(parquet_retail)
        df_dim_vehicle = pd.read_parquet(parquet_dim)

        df_retail = df_retail[df_retail["MOVIMENT_ID"] == 14]
        df_retail['DATE_TIME'] = pd.to_datetime(df_retail['DATE_TIME'])
        df_retail_deduplicated = df_retail.sort_values(['CHASSIS', 'DATE_TIME']).drop_duplicates(['CHASSIS'], keep='last')

        df_retail_enrich = df_retail_deduplicated.merge(
            df_dim_vehicle,
            left_on="CHASSIS",
            right_on="CHASSIS_NBR",
            how="inner"
        )

        df_retail_enrich['YEAR'] = df_retail_enrich['DATE_TIME'].dt.year
        df_retail_enrich['MONTH'] = df_retail_enrich['DATE_TIME'].dt.month

        if 'SEGMENTATION' not in df_retail_enrich.columns:
            df_retail_enrich['SEGMENTATION'] = 'UNKNOWN'

        df = (
            df_retail_enrich
            .groupby(["LINE", "SEGMENTATION", "YEAR", "MONTH"])
            .size()
            .reset_index(name='retail_total')
        )
    else:
        raise FileNotFoundError("Parquet não encontrado. Salve df_retail.parquet e df_dim_vehicle.parquet primeiro.")

# Converter datas
if 'Date' not in df.columns:
    df['day'] = 1
    df['Date'] = pd.to_datetime(df[['YEAR', 'MONTH', 'day']])

# Agrupamento Temporal Mensal (Primeiro dia do mês)
df['date_month'] = df['Date'].dt.to_period('M').dt.to_timestamp()

# ==============================================================================
# 2. CRIAÇÃO DO PAINEL (PANEL DATA)
# ==============================================================================
# Definindo os grupos que formam nossas séries temporais
groups = ['LINE', 'SEGMENTATION']

# Limpeza de Nulos nas Categorias (TFT não aceita NaN em categorias)
for col in groups:
    df[col] = df[col].fillna('UNKNOWN').astype(str)

# Agregação: Soma de vendas por Mês + Linha + Segmentação
print("🔨 Agregando dados em Painel Granular...")
df_panel = df.groupby(['date_month'] + groups)['retail_total'].sum().reset_index(name='sales_qty')

# ==============================================================================
# 3. PREENCHIMENTO DE LACUNAS (ZERO SALES)
# ==============================================================================
import itertools

all_dates = pd.date_range(start=df_panel['date_month'].min(), end=df_panel['date_month'].max(), freq='MS')
unique_groups = df_panel[groups].drop_duplicates().values.tolist()

combinations = list(itertools.product(all_dates, unique_groups))
df_skeleton = pd.DataFrame(combinations, columns=['date_month', 'groups'])

df_skeleton[groups] = pd.DataFrame(df_skeleton['groups'].tolist(), index=df_skeleton.index)
df_skeleton = df_skeleton.drop(columns=['groups'])

data = pd.merge(df_skeleton, df_panel, on=['date_month'] + groups, how='left')
data['sales_qty'] = data['sales_qty'].fillna(0)

print(f"✅ Painel Criado! Linhas totais: {len(data)}")
print(f"Exemplo de Séries: {len(unique_groups)} combinações de Linha/Segmento.")
data.head()

⚠️ Usando DF de exemplo (Certifique-se de carregar seus dados reais antes!)


NameError: name 'df_retail_transformed' is not defined

Enriquecimento Macro (Selic, Dolar, Soja)

In [ ]:
# ==============================================================================
# FUNÇÕES DE DOWNLOAD (MACROECONOMIA)
# ==============================================================================
def get_bcb_data_bypass(code, name, start_date):
    url = f'http://api.bcb.gov.br/dados/serie/bcdata.sgs.{code}/dados?formato=json'
    headers = {'User-Agent': 'Mozilla/5.0 Chrome/58.0.3029.110'}
    try:
        response = requests.get(url, headers=headers, verify=False)
        if response.status_code == 200:
            df = pd.DataFrame(response.json())
            df['data'] = pd.to_datetime(df['data'], dayfirst=True)
            df = df.set_index('data').loc[start_date:]
            return df.resample('MS').mean().rename(columns={df.columns[0]: name})
    except:
        pass
    return pd.DataFrame()

# Download dos dados
start_date = data['date_month'].min().strftime('%Y-%m-%d')
print(f"⏳ Baixando dados macro a partir de {start_date}...")

df_selic = get_bcb_data_bypass(432, 'Selic', start_date)
df_dolar = get_bcb_data_bypass(1, 'Dolar', start_date)

# Soja (Yahoo)
try:
    soy = yf.download('ZS=F', start=start_date, progress=False)
    # Tratamento de versões do yfinance
    if isinstance(soy.columns, pd.MultiIndex): 
        col = soy['Adj Close'] if 'Adj Close' in soy.columns else soy['Close']
    else:
        col = soy[['Adj Close']] if 'Adj Close' in soy.columns else soy[['Close']]
    df_soja = col.resample('MS').mean()
    df_soja.columns = ['Soja_Price']
except:
    print("⚠️ Yahoo falhou. Usando mock para soja.")
    df_soja = pd.DataFrame(index=pd.date_range(start=start_date, periods=len(df_selic)+12, freq='MS'))
    df_soja['Soja_Price'] = 30.0 # Mock

# Merge Macro
df_macro = df_selic.join([df_dolar, df_soja], how='outer')

# Merge Final no Dataset Principal
data = pd.merge(data, df_macro, left_on='date_month', right_index=True, how='left')

# Preenchimento (Forward Fill para gaps, 0 para missing total)
data = data.sort_values(['LINE', 'SEGMENTATION', 'date_month']).ffill().bfill()
print("✅ Dados Macro Integrados.")

Preparação para o Transformer (TimeSeriesDataSet)

In [ ]:
# ==============================================================================
# PREPARAÇÃO DO DATASET PARA O PYTORCH FORECASTING
# ==============================================================================
# 1. Índice de Tempo Inteiro (Obrigatório para TFT)
data['time_idx'] = (data['date_month'].dt.year * 12 + data['date_month'].dt.month)
data['time_idx'] -= data['time_idx'].min()

# 2. Feature Engineering Adicional
data['month_str'] = data['date_month'].dt.month.astype(str) # Sazonalidade Categórica
data['log_sales'] = np.log1p(data['sales_qty']) # Estabiliza a variância (Opcional, mas ajuda)

# 3. Configuração da Janela de Tempo (Opção B - Médio Prazo)
max_prediction_length = 6   # Prever 6 meses
max_encoder_length = 24     # Olhar 24 meses para trás
training_cutoff = data["time_idx"].max() - max_prediction_length

# 4. Definição do TimeSeriesDataSet
training = TimeSeriesDataSet(
    data[lambda x: x.time_idx <= training_cutoff],
    time_idx="time_idx",
    target="sales_qty",
    group_ids=groups, # ['LINE', 'SEGMENTATION'] -> Isso cria o Multi-Série
    
    min_encoder_length=max_encoder_length // 2,
    max_encoder_length=max_encoder_length,
    min_prediction_length=1,
    max_prediction_length=max_prediction_length,
    
    static_categoricals=groups, # Embeddings fixos por série
    time_varying_known_categoricals=["month_str"], # Sazonalidade (sabemos o mês futuro)
    
    # Variáveis Reais (Numéricas)
    time_varying_known_reals=["time_idx"], 
    time_varying_unknown_reals=[
        "sales_qty", 
        "Selic", 
        "Dolar", 
        "Soja_Price"
    ],
    
    # Normalização por Grupo (Crucial para séries com volumes diferentes)
    target_normalizer=GroupNormalizer(
        groups=groups, transformation="softplus"
    ), 
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
)

# Dataloaders
batch_size = 64  # Ajuste para 32 se der erro de memória
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
validation = TimeSeriesDataSet.from_dataset(training, data, predict=True, stop_randomization=True)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=0)

print("✅ TimeSeriesDataSet criado com sucesso!")

Treinamento (Training Loop)

In [ ]:
# ==============================================================================
# TREINAMENTO DO MODELO (TFT)
# ==============================================================================
# Checkpoint Callback (Salva o melhor modelo baseado no erro de validação)
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

checkpoint_callback = ModelCheckpoint(monitor="val_loss", mode="min")
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=5, verbose=False, mode="min")

# Trainer Setup
trainer = pl.Trainer(
    max_epochs=30,           # 30 épocas costuma ser bom para esse tamanho de dado
    accelerator="auto",      # Usa GPU se disponível
    gradient_clip_val=0.1,   # Evita explosão do gradiente
    callbacks=[checkpoint_callback, early_stop_callback],
    enable_model_summary=True,
)

# Inicializa o TFT
tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=0.03,
    hidden_size=16,          # Tamanho da rede (Aumente para 64 se tiver muitos dados)
    attention_head_size=1,   # Cabeças de atenção
    dropout=0.1,
    hidden_continuous_size=8,
    output_size=7,           # 7 Quantis por padrão (incerteza)
    loss=QuantileLoss(),     # Função de perda probabilística
    reduce_on_plateau_patience=4,
)

print(f"🧠 Iniciando treinamento do TFT...")
trainer.fit(
    tft,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)

# Carrega o melhor modelo treinado
best_model_path = trainer.checkpoint_callback.best_model_path
best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path)
print(f"✅ Treinamento concluído. Melhor modelo carregado.")

Visualização e Interpretabilidade

In [ ]:
# ==============================================================================
# AVALIAÇÃO E PREVISÃO
# ==============================================================================
# Fazendo a previsão nos dados de validação (últimos 6 meses)
raw_predictions = best_tft.predict(val_dataloader, mode="raw", return_x=True)

# Função para plotar uma série aleatória do dataset
import random

# Escolhemos alguns exemplos para mostrar (ex: 3 séries diferentes)
for i in range(3):
    idx = random.randint(0, len(raw_predictions.output) - 1)
    
    # Descobrindo qual série é essa (Qual Linha/Segmento?)
    # O Pytorch Forecasting esconde isso no decoder_target, vamos simplificar plotando direto
    
    fig, ax = plt.subplots(figsize=(12, 5))
    best_tft.plot_prediction(raw_predictions.x, raw_predictions.output, idx=idx, add_loss_to_title=True, ax=ax)
    plt.title(f"Previsão Probabilística - Exemplo {i+1} (Intervalo de Confiança)")
    plt.show()

# ==============================================================================
# INTERPRETABILIDADE (Variable Importance)
# ==============================================================================
# O que o modelo aprendeu que é importante?
interpretation = best_tft.interpret_output(raw_predictions.output, reduction="sum")
best_tft.plot_interpretation(interpretation)